In [ ]:
import os
import pandas as pd
from itertools import islice
import torch
from torch.utils.data import DataLoader
from utils.text_metrics import evaluate_all_metrics
from utils.temp_utils import *
from utils.lstm_models import DinoLSTMAttnCaptioner, DinoBiLSTMAttnCaptioner
from utils.chexpert_dataset import CheXpertDataset
from utils.padchest_dataset import PadChestGRDataset

# Data

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

os.chdir(os.path.dirname(os.getcwd()))

CSV_PATH = "Datasets/CheXpertPlus/df_chexpert_plus_240401.csv"
IMG_ROOT = "Datasets/CheXpertPlus/PNG"
TEXT_COL = "section_impression"
PATH_COL = "path_to_image"

IMG_SIZE = 224
MAX_LEN = 64
NUM_BATCH = 8

tf = dino_image_transform(img_size=IMG_SIZE)

ds_train = CheXpertDataset(img_root=IMG_ROOT, csv_path=CSV_PATH, split="train", transform=tf, text_col=TEXT_COL)
ds_valid = CheXpertDataset(img_root=IMG_ROOT, csv_path=CSV_PATH, split="valid", transform=tf, text_col=TEXT_COL)
ds_test = CheXpertDataset(img_root=IMG_ROOT, csv_path=CSV_PATH, split="test", transform=tf, text_col=TEXT_COL)

tokenizer = build_tokenizer_from_labels()
pad_id = tokenizer.pad_token_id
eos_id = tokenizer.eos_token_id
bos_id = tokenizer.bos_token_id
collate_fn = CaptionCollate(tokenizer, pad_id)

train_loader = DataLoader(ds_train, batch_size=NUM_BATCH, shuffle=True, collate_fn=collate_fn)
valid_loader = DataLoader(ds_valid, batch_size=NUM_BATCH, shuffle=False, collate_fn=collate_fn)
test_loader = DataLoader(ds_test, batch_size=NUM_BATCH, shuffle=False, collate_fn=collate_fn)

Using device: cuda
[INFO] Kept 47494/223462 rows with existing PNGs under C:\Users\emman\Desktop\PROYECTOS_VS_CODE\PRUEBAS_DE_PYTHON\CheXpertPlus\PNG
[INFO] Kept 47494/223462 rows with existing PNGs under C:\Users\emman\Desktop\PROYECTOS_VS_CODE\PRUEBAS_DE_PYTHON\CheXpertPlus\PNG
[INFO] Kept 47494/223462 rows with existing PNGs under C:\Users\emman\Desktop\PROYECTOS_VS_CODE\PRUEBAS_DE_PYTHON\CheXpertPlus\PNG


# Model

In [4]:
# DINO ViT-S/16 hidden size is 384 
EMBEDDING_D_IMG = 384
N_PREFIX = (IMG_SIZE // 16) ** 2  # number of visual prefix tokens (including CLS)

model = DinoBiLSTMAttnCaptioner(
    vocab_size=tokenizer.vocab_size,
    d_img=EMBEDDING_D_IMG,
    d_h=512,
    pad_id=pad_id,
    dino_model_id="facebook/dinov3-vits16-pretrain-lvd1689m",
    freeze_dino=True,
).to(device)

# Train Parameters

In [5]:
optimizer = torch.optim.AdamW(
    filter(lambda p: p.requires_grad, model.parameters()), lr=3e-4, weight_decay=1e-2
)
loss = sequence_ce_loss
NUM_EPOCHS = 100
BATCHES_PER_EPOCH = 10

# Training

In [6]:
for epoch in range(NUM_EPOCHS):
    slice_train_loader = islice(train_loader, BATCHES_PER_EPOCH)
    slice_valid_loader = islice(valid_loader, BATCHES_PER_EPOCH)
    train_stats = train_one_epoch(model, slice_train_loader, optimizer, device, pad_id, num_batches=BATCHES_PER_EPOCH, loss_fn=loss, grad_clip=1.0)
    val_stats = evaluate(model, slice_valid_loader, device, pad_id, num_batches=BATCHES_PER_EPOCH, loss_fn=loss)
    print(f"Epoch {epoch + 1}: Train Loss={train_stats['loss']:.4f}, PPL={train_stats['ppl']:.2f} | "
            f"Val Loss={val_stats['val_loss']:.4f}, Val PPL={val_stats['val_ppl']:.2f}")

Training:   0%|          | 0/10 [00:00<?, ?it/s]C:\Users\emman\Desktop\PROYECTOS_VS_CODE\PRUEBAS_DE_PYTHON\Chest-X-ray-Diagnosis-Automated-Reporting-using-CNNs-and-LLMs---UDEM-PEF-Thesis-Fall-2025\utils\temp_utils.py:136: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(dtype=torch.bfloat16):
Evaluating: 100%|██████████| 10/10 [00:04<00:00,  2.24it/s]


Epoch 1: Train Loss=9.5185, PPL=21870.51 | Val Loss=7.6469, Val PPL=2134.68


Evaluating: 100%|██████████| 10/10 [00:04<00:00,  2.29it/s]


Epoch 2: Train Loss=6.8096, PPL=1059.04 | Val Loss=5.9161, Val PPL=390.86


Evaluating: 100%|██████████| 10/10 [00:04<00:00,  2.27it/s]


...

Epoch 98: Train Loss=2.8743, PPL=17.83 | Val Loss=2.8919, Val PPL=18.06


Evaluating: 100%|██████████| 10/10 [00:04<00:00,  2.19it/s]


Epoch 99: Train Loss=2.8839, PPL=18.00 | Val Loss=2.8885, Val PPL=18.00


Evaluating: 100%|██████████| 10/10 [00:04<00:00,  2.17it/s]

Epoch 100: Train Loss=2.8836, PPL=18.03 | Val Loss=2.8940, Val PPL=18.10


# Test Parameters

In [7]:
BATCHES_PER_TEST = 1
GREEDY_DECODE = True
TEST_MAX_LEN = 256
TEST_TOP_P = 0.9
TEST_TEMPERATURE = 0.9

# Test

In [8]:
slice_test_loader = islice(test_loader, BATCHES_PER_TEST)
test_stats = evaluate(model, slice_test_loader, device, pad_id, num_batches=BATCHES_PER_TEST)
print(f"Test Loss={test_stats['val_loss']:.4f}, Test PPL={test_stats['val_ppl']:.2f}")

Evaluating: 100%|██████████| 1/1 [00:00<00:00,  1.94it/s]

Test Loss=2.8456, Test PPL=17.21


# Test Report Generation

In [9]:
with torch.no_grad():
    for pixel_values, ids_loader, paths, raw_labels in test_loader:
        pixel_values = pixel_values.to(device)
        gen_ids = model.generate(
            pixel_values=pixel_values,
            bos_id=bos_id, eos_id=eos_id,
            max_new_tokens=TEST_MAX_LEN, top_p=TEST_TOP_P, temperature=TEST_TEMPERATURE, greedy=GREEDY_DECODE
        )

        info = model.generate_with_logging(
            pixel_values=pixel_values,             # [B, C, H, W]
            bos_id=tokenizer.bos_token_id,
            eos_id=tokenizer.eos_token_id,
            tokenizer=tokenizer,
            preset="safe_sample",
            stop_sequences=None, #["\n\n", "Impression:"],
            max_new_tokens=128,
        )
        print("sequences:", info["sequences"].shape)
        for i, s in enumerate(info["per_sample"]):
            print(f"[{i}] EOS={s['stopping']['hit_eos']} rep={s['repetition']}")
            print(s["text"].get("generated", "")[:200])

        eval_results = evaluate_all_metrics(raw_labels, [s["text"]["generated"] for s in info["per_sample"]], evaluation_mode="CheXagent")
        for metric, scores in eval_results.items():
            print(f"{metric}: {scores}")

        print("Predictions (first batch):")
        for i in range(gen_ids.size(0)):
            text_gen = tokenizer.decode(gen_ids[i].tolist())
            text_tgt = tokenizer.decode(ids_loader[i].tolist())
            print(f"\nGEN {i+1}:", text_gen)
            print(f"TGT {i+1}:", text_tgt)
            results = evaluate_all_metrics([text_tgt], [text_gen], evaluation_mode="CheXagent")
            for metric, scores in results.items():
                print(f"{metric}: {scores}")
        del pixel_values, ids_loader, paths, raw_labels, gen_ids
        torch.cuda.empty_cache()
        break

sequences: torch.Size([8, 129])
[0] EOS=True rep={'max_token_run': 3, 'max_repeat_trigram': 1, 'max_repeat_4gram': 1}
initial initial initial0101017777772
[1] EOS=True rep={'max_token_run': 3, 'max_repeat_trigram': 1, 'max_repeat_4gram': 1}
gradual gradual gradual views views views reveal study study study dated 52 52 522
[2] EOS=True rep={'max_token_run': 3, 'max_repeat_trigram': 1, 'max_repeat_4gram': 1}
sequence sequence sequenceic angles angles angles rotation rotation rotation artifact artifactual the the preceding t t t444 857575 of of treitzzzgan.
[3] EOS=False rep={'max_token_run': 3, 'max_repeat_trigram': 1, 'max_repeat_4gram': 1}
negative negative for confirmationionionion clearly clearly clearly central centrally questioned questioned questioned described described described below below below just just just beyond beyond beyo
[4] EOS=True rep={'max_token_run': 3, 'max_repeat_trigram': 1, 'max_repeat_4gram': 1}
gradual gradual gradual vertebral bodies bodies bodiesll cephalic

In [10]:
# Print number of model parameters
total_params = sum(p.numel() for p in model.parameters())
print(f"Total model parameters: {total_params}")

Total model parameters: 119691252


In [11]:
info

{'per_sample': [{'preset': 'safe_sample',
   'params': {'do_sample': True,
    'temperature': 0.7,
    'top_p': 0.9,
    'top_k': 50,
    'repetition_penalty': 1.15,
    'no_repeat_ngram_size': 3},
   'lengths': {'prompt_tokens': 1, 'new_tokens': 11, 'total_tokens': 12},
   'stopping': {'hit_eos': True, 'eos_pos': 10, 'stop_sequences': []},
   'repetition': {'max_token_run': 3,
    'max_repeat_trigram': 1,
    'max_repeat_4gram': 1},
   'probes': [{'step': 1,
     'entropy': 1.865197777748108,
     'topk': [{'token_id': 18258, 'p': 0.419748455286026},
      {'token_id': 4366, 'p': 0.21426939964294434},
      {'token_id': 3068, 'p': 0.08862230181694031},
      {'token_id': 4954, 'p': 0.0687548890709877},
      {'token_id': 23589, 'p': 0.03689390793442726}]},
    {'step': 2,
     'entropy': -0.0,
     'topk': [{'token_id': 3288, 'p': 1.0},
      {'token_id': 2, 'p': 0.0},
      {'token_id': 0, 'p': 0.0},
      {'token_id': 3, 'p': 0.0},
      {'token_id': 1, 'p': 0.0}]},
    {'step': 3,


In [12]:
text = "1.  STABLE SMALL LEFT INTERNAL JUGULAR OPACITIES WITH PATCHY TUBE AND NASOGASTRIC TUBES, RIGHT LOWER MEDIASTINAL SIDED CATHETER.  NO SIGNIFICANT CHANGE IN THE PREVIOUS STUDYDEMONSTRATE ATELECTASIS O"
encoded = tokenizer.encode(text)
print("BOS token id:", tokenizer.bos_token_id, "EOS token id:", tokenizer.eos_token_id, "PAD token id:", tokenizer.pad_token_id)
print(encoded)

BOS token id: 101 EOS token id: 102 PAD token id: 0
[101, 122, 119, 6111, 1353, 1286, 4422, 34986, 5552, 39280, 49176, 1114, 10085, 1183, 7159, 1105, 9468, 7301, 32519, 11182, 117, 1268, 2211, 2394, 34979, 7050, 11641, 5855, 30682, 119, 1185, 2418, 1849, 1107, 1103, 2166, 2025, 31386, 8756, 18465, 14229, 184, 102]
